# Project: Finding Lane Lines on the Road

Import useful packages

In [1]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math
import pandas as pd
%matplotlib inline

# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

Functions

In [2]:
def grayscale(img):
    """Return an image in grayscale
       To show the image: plt.imshow(gray, cmap='gray')"""
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
def canny(img, low_threshold, high_threshold):
    """Applies the Canny transform"""
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    """Applies a Gaussian Noise kernel"""
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    """
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    `vertices` should be a numpy array of integer points.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=10):
    """
    Function that recives as imput the hough image and extract form it 
    the two lines corresponding to the left and right lane line

    """
    # Define and initialize the global and local variables
    global x_min_left_old
    global x_max_left_old
    global x_min_right_old
    global x_max_right_old

    m_mean_left = 0;
    b_mean_left = 0;
    count_left = 0;
    x_min_left = 0;
    x_max_left = 0;
    
    m_mean_right = 0;
    b_mean_right = 0;
    count_right = 0;
    x_min_right = 0;
    x_max_right = 0;
    
    y_min = 540   # y_min and y_max are defined equal to the ones of the polygon of the region of interest
    y_max = 350 
    
    weight = 0.3 # weight between the old line and the new line detected
    
    ## Left and Right line extraction
    
    for line in lines:
        for x1,y1,x2,y2 in line:
            if  (y2-y1)/(x2-x1) < -0.2: # select only left lines with m < -0.2
                m_mean_left = m_mean_left + (y2-y1)/(x2-x1)          # sum of the left slopes
                b_mean_left = b_mean_left + y1 - (y2-y1)/(x2-x1)*x1  # sum of the left intersects
                count_left = count_left + 1
            elif (y2-y1)/(x2-x1) > 0.2: # select only right lines with m > 0.2
                m_mean_right = m_mean_right + (y2-y1)/(x2-x1)         # sum of the right slopes
                b_mean_right = b_mean_right + y1 - (y2-y1)/(x2-x1)*x1 # sum of the right intersects
                count_right = count_right + 1
                
    if count_left != 0 :
        m_mean_left = m_mean_left/ count_left   # average of the left slopes
        b_mean_left = b_mean_left/ count_left   # average of the left intersects
        x_min_left = int((y_min - b_mean_left)// m_mean_left) # define the x_min coordinate of the left line
        x_max_left = int((y_max - b_mean_left)// m_mean_left) # define the x_max coordinate of the left line
        
        # Weighting function betwn the previous and current x left coordinates (since the y coordinates are fixed)
        if x_min_left_old != 0 and x_max_left_old != 0 : 
            x_min_left = int(weight*x_min_left + (1-weight)*x_min_left_old)
            x_max_left = int(weight*x_max_left + (1-weight)*x_max_left_old)
            
    left_line = cv2.line(img, (x_min_left, y_min), (x_max_left, y_max), color, thickness)
        
    if count_right != 0 :
        m_mean_right = m_mean_right/ count_right # average of the right slopes
        b_mean_right = b_mean_right/ count_right # average of the right intersects
        x_min_right = int((y_min - b_mean_right)// m_mean_right) # define the x_min coordinate of the right line
        x_max_right = int((y_max - b_mean_right)// m_mean_right) # define the x_max coordinate of the right line
        
        # Weighting function betwn the previous and current x right coordinates (since the y coordinates are fixed)
        if x_min_right_old != 0 and x_max_right_old != 0 : 
            x_min_right = int(weight*x_min_right + (1-weight)*x_min_right_old)
            x_max_right = int(weight*x_max_right + (1-weight)*x_max_right_old)
            
    right_line = cv2.line(img, (x_min_right, y_min), (x_max_right, y_max), color, thickness)
    
    # Save the current x coordinates for the next iteration 
    x_min_left_old = x_min_left
    x_max_left_old = x_max_left
    x_min_right_old = x_min_right
    x_max_right_old = x_max_right
    

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    """
    `Function that takes as input the output of a Canny transform 
    (img) and returns an image with hough lines drawn.
    """
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
            
    return line_img

def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + γ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, γ)

Define a function that, taken as imput a single image, execute the whole pipeline for the lane lines detection.

In [3]:
def process_image(image):
    gray_image = grayscale(image);
    canny_image = canny(gray_image, 50, 150)
    track_selection = np.array([[[400,350],[600,350],[900,540],[150,540]]]); 
    track_image = region_of_interest(canny_image, track_selection)
    rho = 2  #distance resolution in pixels of the Hough grid
    theta = np.pi/180 # angular resolution in radians of the Hough grid
    threshold = 10    # minimum number of votes (intersections in Hough grid cell)
    min_line_length = 20 #minimum number of pixels making up a line
    max_line_gap = 5    # maximum gap in pixels between connectable line segments
    hough_image = hough_lines(track_image,rho, theta, threshold, min_line_length, max_line_gap )
    overlapped_image = weighted_img(hough_image, image);
    
    return overlapped_image

# Test on images

Define global variables

In [4]:
x_min_left_old = 0 
x_max_left_old = 0
x_min_right_old = 0
x_max_right_old = 0

Read all the images

In [5]:
raw_image = []
image_name = []

raw_image.append(mpimg.imread('test_images/solidWhiteRight.jpg'))
image_name.append('test_images_output/solidWhiteRight.jpg')

raw_image.append(mpimg.imread('test_images/solidWhiteCurve.jpg'))
image_name.append('test_images_output/solidWhiteCurve.jpg')

raw_image.append(mpimg.imread('test_images/solidYellowCurve.jpg'))
image_name.append('test_images_output/solidYellowCurve.jpg')

raw_image.append(mpimg.imread('test_images/solidYellowCurve2.jpg'))
image_name.append('test_images_output/solidYellowCurve2.jpg')

raw_image.append(mpimg.imread('test_images/solidYellowLeft.jpg'))
image_name.append('test_images_output/solidYellowLeft.jpg')

raw_image.append(mpimg.imread('test_images/whiteCarLaneSwitch.jpg'))
image_name.append('test_images_output/whiteCarLaneSwitch.jpg')

Execute the whole pipeline on the images and save it on 'test_images_output' folder

In [6]:
for i in range(0, len(raw_image)):
    overlapped_image = process_image(raw_image[i])
    plt.imsave(image_name[i],overlapped_image)

# Test on video

Define global variables

In [7]:
x_min_left_old = 0 
x_max_left_old = 0
x_min_right_old = 0
x_max_right_old = 0

Load the first clip, execute the pipeline and save the result into 'test_videos_output' folder

In [8]:
white_output = 'test_videos_output/solidWhiteRight.mp4'
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image) #NOTE: this function expects color images!!
%time white_clip.write_videofile(white_output, audio=False)

t:   2%|█▌                                                                   | 5/221 [00:00<00:04, 49.15it/s, now=None]

Moviepy - Building video test_videos_output/solidWhiteRight.mp4.
Moviepy - Writing video test_videos_output/solidWhiteRight.mp4



Moviepy - Done !
Moviepy - video ready test_videos_output/solidWhiteRight.mp4
Wall time: 4.3 s


Load the second clip, execute the pipeline and save the result into 'test_videos_output' folder

In [9]:
yellow_output = 'test_videos_output/solidYellowLeft.mp4'
##clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4').subclip(0,5)
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

t:   1%|▌                                                                    | 6/681 [00:00<00:11, 59.57it/s, now=None]

Moviepy - Building video test_videos_output/solidYellowLeft.mp4.
Moviepy - Writing video test_videos_output/solidYellowLeft.mp4



Moviepy - Done !
Moviepy - video ready test_videos_output/solidYellowLeft.mp4
Wall time: 13.7 s


Open an see the output of the lane detection.

In [10]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

In [12]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))